In [35]:
# !pip install gender_guesser
# !pip install missingno
# !pip install wordcloud

In [8]:
import pandas as pd
import gender_guesser.detector as gender
from sklearn import preprocessing
import random
import nltk
import re
import string
import json
import urllib.request
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from textblob import TextBlob
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from textblob import Word


#  Yelp Business Reviews Data 

In [9]:
import glob
import re
import pandas as pd

def Reading_Data(DataFrame, path):
    for fname in glob.glob(path):
        df = pd.read_csv(fname)
        DataFrame = DataFrame.append(df, ignore_index=True)
        DataFrame=DataFrame.drop_duplicates(keep="first",subset=["Customer_Review","Customer_Review_Date"])
#         DataFrame['Category']=((fname.split("_")[-1]).split(".")[0])
    return DataFrame

if __name__ == '__main__':
    df = pd.DataFrame()
    path = '../Data_Business_Reviews/Business_Reviews_Seattle_withoutDuplicates.csv'
   
    Original_DataFrame = Reading_Data(df, path)

In [10]:
Original_DataFrame.shape

(97584, 26)

In [11]:
Original_DataFrame.head()

,Unnamed: 0,Business_Name,Business_Address,Business_ReviewCount,Business_Rating,Business_Photos_Count,Business_Timings,Business_Claim_status,Customer_Name,Customer_Friends_count,...,Customer_Review_Date,Customer_Review_Uploaded_Photos,Customer_Review_Useful,Customer_Review_Funny,Customer_Review_Cool,Business_response_By,Business_response_Date,Business_Response_for_Review,Business_Response,Category
0,0,Britsport of Seattle,"4214 24th Ave WSeattle, WA 98199",1,5.0,Null,NaN,Null,Walt F.,0,...,9/16/2019,First,0,0,0,Null,Null,Null,0,AutoRepair
1,1,Service King Collision West Seattle,"4501 38th Ave SWSeattle, WA 98126",31,4.0,Null,7:00 AM - 6:00 PM,Claimed,Stephanie V.,13,...,2/15/2021,0,0,0,0,Null,Null,Null,0,AutoRepair
2,2,Service King Collision West Seattle,"4501 38th Ave SWSeattle, WA 98126",31,4.0,Null,7:00 AM - 6:00 PM,Claimed,Diane R.,7,...,9/4/2020,0,0,0,0,Null,Null,Null,0,AutoRepair
3,3,Service King Collision West Seattle,"4501 38th Ave SWSeattle, WA 98126",31,4.0,Null,7:00 AM - 6:00 PM,Claimed,Roger B.,373,...,12/18/2019,0,0,0,0,Business Customer Service,12/20/2019,"Roger, we're sorry to hear about your car but ...",1,AutoRepair
4,4,Service King Collision West Seattle,"4501 38th Ave SWSeattle, WA 98126",31,4.0,Null,7:00 AM - 6:00 PM,Claimed,Dan K.,0,...,9/16/2018,0,3,0,0,Business Customer Service,9/17/2018,We're deeply sorry to hear about your unpleasa...,1,AutoRepair


In [12]:
# import missingno as msno

# msno.heatmap(DataFrame)


In [13]:
Original_DataFrame.isna().sum()

Unnamed: 0                             0
Business_Name                          0
Business_Address                       0
Business_ReviewCount                   0
Business_Rating                        0
Business_Photos_Count                  0
Business_Timings                   16555
Business_Claim_status                  0
Customer_Name                          0
Customer_Friends_count                 0
Customer_Reviews_count                 0
Customer_Photos_count                  0
Customer_Elite                         0
Customer_Elite_Year                    0
Customer_Rating                        0
Customer_Review                        0
Customer_Review_Date                   0
Customer_Review_Uploaded_Photos        0
Customer_Review_Useful                 0
Customer_Review_Funny                  0
Customer_Review_Cool                   0
Business_response_By                   0
Business_response_Date                 0
Business_Response_for_Review           0
Business_Respons

In [14]:
Original_DataFrame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97584 entries, 0 to 97593
Data columns (total 26 columns):
Unnamed: 0                         97584 non-null int64
Business_Name                      97584 non-null object
Business_Address                   97584 non-null object
Business_ReviewCount               97584 non-null int64
Business_Rating                    97584 non-null float64
Business_Photos_Count              97584 non-null object
Business_Timings                   81029 non-null object
Business_Claim_status              97584 non-null object
Customer_Name                      97584 non-null object
Customer_Friends_count             97584 non-null int64
Customer_Reviews_count             97584 non-null int64
Customer_Photos_count              97584 non-null int64
Customer_Elite                     97584 non-null object
Customer_Elite_Year                97584 non-null object
Customer_Rating                    97584 non-null int64
Customer_Review                    97584 

# Data Cleaning

In [45]:
# Removing the street address from the address and extracting city and zipcode seperately
def get_state(address):
#     print(address)
    state = address.split(" ")[-2]
    return state
def get_zipcode(address):
    zipcode = address.split()[-1]
    return zipcode
def get_city(address):
    temp =(address.split(',')[0])
    city =([s for s in re.split("([A-Z][^A-Z]*)", temp) if s ][-1])
    return city

# Business_Timings  
def Handle_timings_from(time):        
    if '-' in str(time):
        time_from =str(time).split('-')[0]
        time_from= time_from.split(':')[0]   
        return time_from
    
    else:
        time_from = ''.join(filter(str.isdigit, str(time)))
        return time_from

def Handle_timings_to(time):    
    if '-' in str(time):
        time_to =str(time).split('-')[1]
        time_to= time_to.split(':')[0]  
        return time_to
    else:
        time_to = ''.join(filter(str.isdigit, str(time)))
        return time_to
      
def Operational_hours(time):
    time = str(time)
    
    if not time:
        operational_hours = 0
        return operational_hours
    
    if 'Next day' in time:
        operational_hours = 24
        return operational_hours
    
    if '24' in time:
        operational_hours = 24
        return operational_hours    
    
    if '-' in (time):
        time_from =(time).split('-')[0]
        time_to = (time).split('-')[1]
        time_from= time_from.split(':')[0]        
        time_to= time_to.split(':')[0]    
        operational_hours = (int(time_to) + 12) - int(time_from)
        return operational_hours
    
# Converting the Customer_Name column to gender based on names
def Converting_Names_to_gender(name):
    name =str(name).split()[0]
    name = re.sub("[^a-zA-Z]+", "", name)
    d = gender.Detector()
    gender_type = d.get_gender(name)
    return gender_type


def Remove_spam_gender(gender):
    valid = ['mostly_female', 'female', 'male', 'unknown','mostly_male']    
    if gender not in valid:
        gender = 'unknown'
    if gender == 'mostly_female':
        gender ='female'
    if gender == 'mostly_male':
        gender ='male'
        
    return gender

# function to remove the unwanted text in Date column 
def cleaning_Business_date(date):
    date = str(date).split('U')[0]
    return date

# function to handle the date column in the data frame
def Handling_Date_Feature(date):
    df = pd.Timestamp(date)
#     df.weekday_name   
    day = df.dayofweek
    return day


In [46]:
# copy the Original_DataFrame to Cleaned_Dataframe
Cleaned_DataFrame = Original_DataFrame.copy()
Cleaned_DataFrame = Cleaned_DataFrame.head(100)

In [47]:
Cleaned_DataFrame.shape

(100, 25)

In [48]:
# Address
Cleaned_DataFrame['Business_State'] = Cleaned_DataFrame.apply(lambda row: get_state(row['Business_Address']), axis=1) 
Cleaned_DataFrame['Business_Zipcode'] = Cleaned_DataFrame.apply(lambda row: get_zipcode(row['Business_Address']), axis=1)
Cleaned_DataFrame['Business_City'] = Cleaned_DataFrame.apply(lambda row: get_city(row['Business_Address']), axis=1) 
Cleaned_DataFrame['Business_Zipcode'] = Cleaned_DataFrame['Business_Zipcode'].replace(to_replace ='WA', value =0)
Cleaned_DataFrame['Business_Zipcode'] = Cleaned_DataFrame['Business_Zipcode'].apply(lambda s: str(s).replace('"', ""))

# Handling Timings column and replacing na values in Business_Timings_From  with 0
Cleaned_DataFrame['Business_Timings_From']= Cleaned_DataFrame.apply(lambda row: Handle_timings_from(row['Business_Timings']), axis=1)
Cleaned_DataFrame['Business_Timings_From'] = Cleaned_DataFrame['Business_Timings_From'] .replace(to_replace ='', value =0)

# Handling Timings column and replacing na values in Business_Timings_to  with 0
Cleaned_DataFrame['Business_Timings_To']= Cleaned_DataFrame.apply(lambda row: Handle_timings_to(row['Business_Timings']), axis=1)
Cleaned_DataFrame['Business_Timings_To'] = Cleaned_DataFrame['Business_Timings_To'].replace(to_replace ='', value =0)

# Deriving Operational Hours
Cleaned_DataFrame['Business_Operational_Hours']= Cleaned_DataFrame.apply(lambda row: Operational_hours(row['Business_Timings']), axis=1)


# Business_Claim_status
Cleaned_DataFrame['Business_Claim_status'] = Cleaned_DataFrame['Business_Claim_status'].replace(to_replace ='Null', value =0)
Cleaned_DataFrame['Business_Claim_status'] = Cleaned_DataFrame['Business_Claim_status'].replace(to_replace ='Claimed', value =1)

# Customer Gender
Cleaned_DataFrame['Customer_Gender'] = Cleaned_DataFrame.apply(lambda row: Converting_Names_to_gender(row['Customer_Name']), axis=1)
Cleaned_DataFrame['Customer_Gender'] = Cleaned_DataFrame.apply(lambda row: Remove_spam_gender(row['Customer_Gender']), axis=1)


# Customer_Uploaded_Photos
Cleaned_DataFrame['Customer_Review_Uploaded_Photos'] = Cleaned_DataFrame['Customer_Review_Uploaded_Photos'].replace(to_replace =['First','ROTD','Purchased','Booked'], value =-1)

# Updating the Date Columns
Cleaned_DataFrame['Customer_Review_Date'] = Cleaned_DataFrame.apply(lambda row: cleaning_Business_date(row['Customer_Review_Date']), axis=1)
# Updating the Date Columns
Cleaned_DataFrame['Customer_Review_Day'] = Cleaned_DataFrame.apply(lambda row: Handling_Date_Feature(row['Customer_Review_Date']), axis=1)

In [49]:
# Remove the unwanted Column
Cleaned_DataFrame.drop('Unnamed: 0', axis=1, inplace=True)
Cleaned_DataFrame.drop('Business_response_By', axis=1, inplace=True)
Cleaned_DataFrame.drop('Business_response_Date', axis=1, inplace=True)
Cleaned_DataFrame.drop('Business_Response_for_Review', axis=1, inplace=True)
Cleaned_DataFrame.drop('Business_Photos_Count', axis=1, inplace=True)
Cleaned_DataFrame.drop('Customer_Elite_Year', axis=1, inplace=True)
# Cleaned_DataFrame.drop('Business_Address', axis=1, inplace=True)
# Cleaned_DataFrame.drop('Business_Timings', axis=1, inplace=True)
# Cleaned_DataFrame.drop('Customer_Name', axis=1, inplace=True)

In [50]:
Cleaned_DataFrame.head()

,Business_Name,Business_ReviewCount,Business_Rating,Business_Claim_status,Customer_Friends_count,Customer_Reviews_count,Customer_Photos_count,Customer_Elite,Customer_Rating,Customer_Review,...,Customer_Review_Cool,Business_Response,Business_State,Business_Zipcode,Business_City,Business_Timings_From,Business_Timings_To,Business_Operational_Hours,Customer_Gender,Customer_Review_Day
0,All Covered Painting,21,4.5,1,51,36,0,No,5,I needed a ceiling skim coated to cover thick ...,...,0,0,WA,98102,Seattle,8,5,9,female,3
1,All Covered Painting,21,4.5,1,0,30,1,No,5,We had All Covered Painting power wash & stain...,...,0,0,WA,98102,Seattle,8,5,9,male,1
2,All Covered Painting,21,4.5,1,0,47,3,No,5,I hired All Covered Painting to paint my kitch...,...,0,0,WA,98102,Seattle,8,5,9,female,1
3,All Covered Painting,21,4.5,1,0,1,0,No,5,we tend to be a little detail oriented and had...,...,0,0,WA,98102,Seattle,8,5,9,male,1
4,All Covered Painting,21,4.5,1,218,7,0,No,5,We just had All Covered Painting paint our hom...,...,0,0,WA,98102,Seattle,8,5,9,female,1


## Applying Sentiment Analysis on Reviews

In [51]:
# Function to clean the text
def text_cleaning(review):    
    review = review.lower()
    review = review.strip()
    review = re.sub(r'[0-9]', '', review)   
    review = "".join([word for word in review if word not in string.punctuation])
    # spelling correct
    review = TextBlob(review).correct()
    # Lemmatize final review format
    review =  "".join([Word(word).lemmatize() for word in review])
    
    return review
    
def avg_word(review):
    words = review.split()
    return (sum(len(word) for word in words) / len(words))

# negative, nautral, positive analysis
def getTextAnalysis(a):
    if a < 0:
        return "Negative"
    elif a == 0:
        return "Neutral"
    else:
        return "Positive"



In [52]:
# Text cleaning of customer Review
Cleaned_DataFrame['Cleaned_Customer_Review'] = Cleaned_DataFrame.apply(lambda row: text_cleaning(row['Customer_Review']), axis=1)
# Calculate word count
Cleaned_DataFrame['review_word_count'] = Cleaned_DataFrame['Customer_Review'].apply(lambda x: len(str(x).split(" ")))
# Calculate character count
Cleaned_DataFrame['review_char_count'] = Cleaned_DataFrame['Customer_Review'].str.len()
# Calculate average words
Cleaned_DataFrame['review_avg_word_length'] = Cleaned_DataFrame['Customer_Review'].apply(lambda x: avg_word(x))
# Calculating the Review sentence count
Cleaned_DataFrame['review_sentence_count'] = Cleaned_DataFrame['Customer_Review'].apply(lambda x: len(str(x).split(".")))
# Calculating the Review average sentence length
Cleaned_DataFrame['review_avg_sentence_length'] = Cleaned_DataFrame['review_word_count']/Cleaned_DataFrame['review_sentence_count']
# Calculate polarity
Cleaned_DataFrame['polarity'] = Cleaned_DataFrame['Cleaned_Customer_Review'].apply(lambda x: TextBlob(x).sentiment[0])
# Calculate subjectivity
Cleaned_DataFrame['subjectivity'] = Cleaned_DataFrame['Cleaned_Customer_Review'].apply(lambda x: TextBlob(x).sentiment[1])
# Applying Stenitment score
Cleaned_DataFrame['Sentiment_Score'] = Cleaned_DataFrame.apply(lambda row: getTextAnalysis(row['polarity']), axis=1)


In [53]:
Cleaned_DataFrame[[ 'review_word_count',  'review_char_count', 'review_avg_word_length', 'polarity' , 'subjectivity','Sentiment_Score']].head()

,review_word_count,review_char_count,review_avg_word_length,polarity,subjectivity,Sentiment_Score
0,150,797,4.320000,0.122454,0.554630,Positive
1,53,300,4.679245,0.371429,0.807143,Positive
2,107,625,4.841121,0.137121,0.383333,Positive
3,96,493,4.145833,0.127604,0.516667,Positive
4,72,410,4.625000,0.383636,0.621212,Positive


In [25]:
Cleaned_DataFrame.to_csv('../Code_Machine_Learning_Model/clean_dataframe.csv')